In [2]:
import pandas as pd
from pathlib import Path
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.combine import SMOTEENN
import pickle

## Import data from s3.amazonaws

In [3]:
data_2007_2020 = 'https://s3.amazonaws.com/parkerhiggins-nba-draft-bucket/07-20_MBB_StatsAndDraft.csv'

raw_df = pd.read_csv(data_2007_2020)

## Preprocessing: Remove non-numerical columns

In [4]:
numerical_cols = raw_df.dtypes[raw_df.dtypes != 'object'].index.tolist()

numerical_df = raw_df[numerical_cols]
numerical_df.index = raw_df['#']

In [5]:
numerical_df['draft_status'] = (numerical_df['season_year']==numerical_df['draft_year']).astype(bool)

C:\Users\brady\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## Preprocessing: Select common individual basketball statistics to use as features in the model

In [6]:
numerical_df.columns

Index(['#', 'gp', 'mpg', 'fgm', 'fga', 'FG%', '3PM', '3PA', '3P%', 'ftm',
       'fta', 'FT%', 'tov', 'pf', 'orb', 'drb', 'rpg', 'apg', 'spg', 'bpg',
       'ppg', 'season_year', 'pk', 'draft_year', 'draft_status'],
      dtype='object')

In [52]:
all_players_df.sort_values('#')
# all_players_df.head()

,#,gp,mpg,fgm,fga,FG%,3PM,3PA,3P%,ftm,...,rpg,apg,spg,bpg,ppg,season_year,pk,draft_year,draft_status,player
player,,,,,,,,,,,,,,,,,,,,,
Stephen Curry,1,34,33.7,9.2,20.2,0.455,3.8,9.9,0.387,6.5,...,4.4,5.6,2.5,0.2,28.6,2009,7.0,2009.0,True,NaN
Markus Howard,1,29,33.2,8.2,19.3,0.422,4.2,10.1,0.412,7.3,...,3.5,3.3,0.9,0.0,27.8,2020,NaN,NaN,False,NaN
Tyler Harvey,1,32,36.9,7.2,15.3,0.469,4.0,9.3,0.431,4.7,...,3.6,2.6,1.1,0.1,23.1,2015,51.0,2015.0,True,NaN
Marcus Keene,1,32,36.8,9.8,21.9,0.447,3.9,10.6,0.368,6.5,...,4.5,4.9,0.8,0.0,30.0,2017,NaN,NaN,False,NaN
Reggie Hamilton,1,36,35.6,7.8,17.5,0.447,3.3,7.8,0.420,7.3,...,3.8,5.1,2.0,0.1,26.2,2012,NaN,NaN,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cody Riley,2000,30,17.5,2.1,4.4,0.474,0.2,0.7,0.300,1.1,...,4.1,0.7,0.3,0.4,5.5,2019,NaN,NaN,False,NaN
Jahcobi Neath,2000,30,18.4,1.8,4.3,0.422,0.4,0.8,0.480,1.3,...,1.9,2.2,0.5,0.0,5.3,2020,NaN,NaN,False,NaN
C.J. Neumann,2000,32,29.4,1.8,3.7,0.504,0.0,0.1,0.000,1.4,...,7.0,1.2,0.8,0.3,5.1,2016,NaN,NaN,False,NaN


In [40]:
all_players_df = numerical_df
all_players_df['player'] = raw_df['player']
all_players_df.to_json('./all_player_stats.json')

C:\Users\brady\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


ValueError: DataFrame index must be unique for orient='columns'.

In [55]:
selected_features = ['ppg','rpg','apg','spg','tov','draft_status','FG%','3P%','FT%']
data_df = numerical_df[selected_features]

X = data_df.drop(columns=['draft_status'])
y = data_df['draft_status']


,ppg,rpg,apg,spg,tov,fgPercent,threePointPercent,freeThrowPercent
player,,,,,,,,
Blake Griffin,14.8,9.2,1.8,1.0,2.3,0.573,0.000,0.595
Anthony Davis,14.2,10.4,1.2,1.4,1.0,0.623,0.150,0.709
Derrick Rose,14.9,4.5,4.7,1.2,2.7,0.477,0.337,0.712
Blake Griffin,22.7,14.4,2.3,1.1,3.3,0.655,0.333,0.588
Anthony Bennett,16.1,8.1,1.0,0.7,1.9,0.533,0.375,0.701
...,...,...,...,...,...,...,...,...
Marcus Damas,11.1,5.1,1.2,0.5,1.7,0.432,0.350,0.730
Marcus Burton,11.0,2.0,1.6,0.6,1.5,0.419,0.373,0.868
Gary Bell,11.0,2.9,1.7,1.0,1.0,0.476,0.427,0.786


## Preprocessing: Use SMOTEENN to oversample drafted players and undersample undrafted players

In [8]:
smoteenn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smoteenn.fit_resample(X,y)

## Preprocessing: Split into training and test datasets

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled,random_state=1,test_size=0.05)

# Logistic Regression Model

In [57]:
lr = LogisticRegression(solver='lbfgs',random_state=1, max_iter=1000)
lr.fit(X_train,y_train)

lr_pred = lr.predict(X_test)

lr_acc_score = accuracy_score(y_test,lr_pred)
lr_matrix = confusion_matrix(y_test,lr_pred)
lr_results = pd.DataFrame({"Prediction": lr_pred, "Actual": y_test}).reset_index(drop=True)
lr_report = classification_report(y_test,lr_pred,zero_division=True)

print(lr_acc_score,'\n')
print(lr_matrix,'\n')
print(lr_report,'\n')

model_filename = 'nba_LogReg_model.sav'
pickle.dump(lr, open(model_filename, 'wb'))

0.8697378872120731 

[[ 992  186]
 [ 142 1198]] 

              precision    recall  f1-score   support

       False       0.87      0.84      0.86      1178
        True       0.87      0.89      0.88      1340

    accuracy                           0.87      2518
   macro avg       0.87      0.87      0.87      2518
weighted avg       0.87      0.87      0.87      2518
 



# Predict 2021 Draft Picks

In [58]:
nba_2021_url = 'https://parkerhiggins-nba-draft-bucket.s3.amazonaws.com/season_stats_data_2020-2021.csv'

raw_2021_data = pd.read_csv(nba_2021_url)

In [59]:
raw_2021_data.columns

Index(['#', 'Player', 'Team', 'GP', 'MPG', 'FGM', 'FGA', 'FG%', '3PM', '3PA',
       '3P%', 'FTM', 'FTA', 'FT%', 'TOV', 'PF', 'ORB', 'DRB', 'RPG', 'APG',
       'SPG', 'BPG', 'PPG', 'Season_Year'],
      dtype='object')

In [60]:
raw_2021_data.head()

,#,Player,Team,GP,MPG,FGM,FGA,FG%,3PM,3PA,...,TOV,PF,ORB,DRB,RPG,APG,SPG,BPG,PPG,Season_Year
0,1,Max Abmas,ORU,28,37.0,8.0,16.8,0.478,3.6,8.3,...,2.3,2.2,0.4,2.8,3.2,3.9,1.5,0.2,24.6,2021
1,2,Luka Garza,UI,31,31.6,9.1,16.4,0.553,1.4,3.2,...,1.5,2.3,3.0,5.7,8.7,1.7,0.7,1.6,24.1,2021
2,3,Antoine Davis,UDM,22,38.5,8.1,19.1,0.424,3.8,10.1,...,3.3,1.6,0.2,2.7,2.9,4.8,1.5,0.0,24.0,2021
3,4,Cam Thomas,LSU,29,33.9,7.0,17.2,0.406,2.3,7.2,...,1.7,1.4,0.6,2.8,3.4,1.4,0.9,0.2,23.0,2021
4,5,A.J. Green,UNI,3,36.3,8.7,18.7,0.464,3.7,9.0,...,3.7,2.0,1.0,4.7,5.7,2.7,1.3,0.7,22.3,2021


In [61]:
numerical_cols_2021 = raw_2021_data.dtypes[raw_2021_data.dtypes != 'object'].index.tolist()

numerical_2021 = raw_2021_data[numerical_cols_2021]
numerical_2021.index = raw_2021_data['#']

selected_features_2021 = ['PPG','RPG','APG','SPG','TOV','FG%','3P%','FT%']

X_2021 = numerical_2021[selected_features_2021]

In [62]:
pred_2021 = lr.predict(X_2021)

In [63]:
draft_prob = lr.predict_proba(X_2021)

draft_prob_true = []
for prob in draft_prob:
    draft_prob_true.append(prob[1])

In [64]:
prediction_display_features = ['Player','Team']
for feature in selected_features_2021:
    prediction_display_features.append(feature)

draft_predictions_2021 = raw_2021_data[prediction_display_features]
draft_predictions_2021['draft_prediction'] = pred_2021
draft_predictions_2021['draft_prob'] = draft_prob_true

C:\Users\brady\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\brady\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [72]:
draft_picks_2021 = draft_predictions_2021.sort_values(by='draft_prob',ascending=False).head(60).drop(['draft_prob','draft_prediction'],axis=1)

draft_picks_2021 = draft_picks_2021.rename(columns={'FG%':'fgPercent','3P%':'threePointPercent','FT%':'freeThrowPercent'})

In [73]:
predictions_filepath = './2021_predictions.json'
draft_picks_2021.to_json(predictions_filepath)

In [74]:
draft_picks_2021.head()

,Player,Team,PPG,RPG,APG,SPG,TOV,fgPercent,threePointPercent,freeThrowPercent
1,Luka Garza,UI,24.1,8.7,1.7,0.7,1.5,0.553,0.440,0.709
8,Terry Taylor,APSU,21.6,11.1,1.6,1.2,2.7,0.521,0.279,0.794
45,Drew Timme,GU,19.0,7.0,2.3,0.7,1.8,0.655,0.286,0.696
46,Kendric Davis,SMU,19.0,4.2,7.6,1.6,2.4,0.481,0.373,0.833
49,Hayden Brown,CIT,18.8,10.5,2.8,1.1,2.5,0.508,0.286,0.704


In [75]:
X.columns

Index(['ppg', 'rpg', 'apg', 'spg', 'tov', 'FG%', '3P%', 'FT%'], dtype='object')